In [ ]:
import os
import pandas as pd
from loguru import logger
import geopandas as gpd
from glob import glob
from rasterstats import zonal_stats
from datetime import datetime
import xarray as xr
import rasterio
import numpy as np
from shapely.geometry import mapping
import rioxarray as rxr

#set up the main function to process the data
def process_variable(nc_var, varfiles, nuts, out_dir):
    logger.info(f'Loading {len(varfiles)} {nc_var} files...')
    
    #open the file
    ds = xr.open_mfdataset(varfiles)
    logger.info(f'Resampling {nc_var} by month...')
    #resample the file to the monthly average
    ds = ds.resample(time='M').mean()
    
    #check if the file contains crs data
    #in case not, add it
    if ds.rio.crs is None:
        ds.rio.write_crs(4326, inplace=True)

    #clip the data to the NUTS shapefile    
    ds_c = ds.rio.clip(nuts.geometry.apply(mapping), nuts.crs, all_touched = True)
    
    #select the time format, the lon and the lat
    time_var, lat_var, lon_var = 'time', 'lat', 'lon'
    time_format = "%Y-%m-%dT%H:%M:%S.%f000"
    
    lats, lons = ds_c[lat_var].values, ds_c[lon_var].values
    #create an output table with the NUTS data
    output_tbl = pd.DataFrame(nuts)
    
    for t, time_step in enumerate(ds_c[time_var].values):
        # for each time step, do the following
        
        data = ds_c[nc_var][t, :, :].values
        datetime_step = datetime.strptime(str(time_step), time_format)
        date = datetime_step.strftime('%Y-%m-%d')
        
        # save the file as a tif
        filename = f'{datetime_step:%Y-%m}_{nc_var}.tif'
        filename = os.path.join(out_dir, filename)
        
        #create a raster files for each time step
        save_raster(filename, data, lons, lats)
        
        #perform zonal statistic
        with rasterio.open(filename) as src:
            affine = src.transform
            array = src.read(1)
            
            ds_zonal_stats = calculate_zonal_stats(nuts, array, affine)
            output_tbl[date] = ds_zonal_stats
            
            if t == 0:
                output_tbl = output_tbl.loc[:,['NUTS_ID', date]]

    return output_tbl

# Create a raster file using rasterio
def save_raster(filename, data, lons, lats):
    with rasterio.open(
        filename,
        'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs='epsg:4326',
        transform=rasterio.transform.from_bounds(lons.min(), lats.min(), lons.max(), lats.max(), data.shape[1], data.shape[0])
    ) as dst:
        dst.write(data, 1)

#perform  zonal statistic
def calculate_zonal_stats(nuts, array, affine):
    ds_zonal_stats = zonal_stats(nuts, array, affine=affine, stats=['sum'], all_touched = True)
    return [i['sum'] for i in ds_zonal_stats]

workflow_folder = 'C:/Users/artuso/Documents/CLIMAXX/preprocessing/New-folder'
# debug if folder does not exist - issue an error to check path

nc_directory = "P:/watxene/ISIMIP/ISIMIP3a/InputData/climate/obsclim_updated/GSWP3-W5E5"

# create outputs folder
if not os.path.exists(os.path.join(workflow_folder, 'data')):
    os.makedirs(os.path.join(workflow_folder, 'data'))
out_dir = 'C:/Users/artuso/Documents/CLIMAXX/preprocessing/New-folder/data2'
nc_vars = ['pr']

os.chdir(nc_directory)
ncs = glob('*.nc')

logger.info(f'Found {len(ncs)} files.')

# select polygons if needed (e.g. 3 regions in Italy)
#regions = ['ITF1', 'ITF2', 'ITF3']
shapefile = os.path.join(workflow_folder, "NUTS_RG_20M_2021_4326.shp")
nuts = gpd.read_file(shapefile)
#filter the NUTS shapefile to the region of interest
#nuts = nuts.loc[nuts['NUTS_ID'].isin(regions)]

#set up the output file
output_tbl = pd.DataFrame(nuts)
#drop the columns in excess
output_tbl = output_tbl.drop(['geometry'], axis = 1)
output_tbl = output_tbl.loc[:,['NUTS_ID']]


for nc_var in nc_vars:
    # For every nc_var contained in the list nc_vars - do the following:
    #if does not work install dependencies netcdf4, h5netcdf and dask

    # specify the files format
    nc_file_pattern = f'gswp3-w5e5_obsclim_{nc_var}_global_daily_'

    # select each of the files that correspond with the format specified
    varfiles = [f for f in ncs if nc_file_pattern in f]
    
    # process the data
    result_tbl = process_variable(nc_var, varfiles, nuts, out_dir)
    
    # merge the results in the output table
    output_tbl = pd.merge(output_tbl, result_tbl, on='NUTS_ID')

# output_tbl_T = output_tbl.set_index('NUTS_ID').transpose()
# function to transpose the dataframe to the required format (columns NUTS_ID, rows YYYY-MM-D)
def transpose_dataframe(gdf, agg_field='NUTS_ID'):
    df = pd.DataFrame(gdf)
        
    df.rename(columns={agg_field: 'timing'}, inplace=True)
    
    df = df.T
    new_header = df.iloc[0] 
    df = df[1:]
    df.columns = new_header 
    
    return df

output_tbl_T = transpose_dataframe (output_tbl)
#save the table as a *csv file
output_tbl_T.to_csv(os.path.join(out_dir, 'precip_table.csv'), index=True)